# DFT Embedding with Qiskit-nature

Nous adoptons le workflow suivant:
1. Effectuer les calculs mean-field (les calculs de référence)
2. Obtenir le Hamiltonien complet du système hors de l'instance `ElectronicStructureProblem`, à partir des intégrales à 1- et 2-corps
3. Définir un espace actif en utilisant `ActiveSpaceTransformer` et en déduire l'Hamiltonien réduit et l'instance `ElectronicStructureProblem` associé
4. Effectuer le mapping dans l'espace qubit
5. Effectuer le calculs des états excités à partir du solver choisi.

De facon plus détaillée on a 

1. **Reference Calculation**

    `driver.run_pyscf()` effectue le calcul Hartree-Fock ou DFT, afin d'obtenir les données nécessaires à la construction de l'Hamiltonien.

2. **Hamiltonian Construction**

    `hamiltonian = ElectronicEnergy(...)` crée un objet `ElectronicEnergy`, représentant l'Hamiltonien électronique.

    `ElectronicIntegrals(...)` est utilisé pour construire les intégrales, cad les termes à 1-électron (+-) et à 2-électrons (++--).

    `S8Integrals(...)` peut-être utilisé pour calculer les intégrales à 2-électrons sous une forme symétrique (format S8).
    
    Cependant, il est plus aisé d'utiliser une instance QCSchema.
    
    ```python
    from qiskit_nature.second_q.formats.qcschema import QCSchema
    from qiskit_nature.second_q.formats import qcschema_translator

    qcschema = driver.to_qcschema(include_dipole=True)
    hamiltonian = qcschema_translator._get_mo_hamiltonian_direct(qcschema)
    ```


3. **Active Space Preparation**

    `active_orbitals` qui spécifie l'ensemble des orbitales qui seront prises en compte dans l'espace actif.

    `ActiveSpaceTransformer` initialise avec le nombre d'orbitales spatiales et d'électrons dans l'espace actif.

    `transformer.prepare_active_space(...)` configure le transformateur en fonction des orbitales actives sélectionnées.

4. **Hamiltonian Reduction**

    `reduced_hamiltonian = transformer.transform_hamiltonian(hamiltonian)` transforme l'Hamiltonien complet du système en un Hamiltonien de l'espace actif, réduisant ainsi la taille du problème.

5. **ElectronicStructureProblem**

    `ElectronicStructureProblem` crée une instance Problem avec `reduced_hamiltonian`.

    Renseigner, pour le Problem, d'autres propriétés pertinentes (comme les énergies orbitales et l'énergie de référence) .

6. **Qubit Mapping and Reduction**

    `ParityMapper` est utilisé pour mapper les opérateurs fermioniques aux opérateurs qubit et on effectue la réduction $\mathbb{Z}_2$.

7. **Excited States Calculation**

    `NumPyEigensolver` est utilisé pour résoudre l'Hamiltonien qubit afin d'obtenir les états excités.

    `ExcitedStatesEigensolver` est configuré pour utiliser `NumPyEigensolver`.

In [ ]:
# !export OMP_NUM_THREADS=12

## Mean Fields calculations

In [2]:
# setup driver
from qiskit_nature.second_q.drivers import MethodType, PySCFDriver
omega = 1.0
driver = PySCFDriver(
    # atom="O 0.0 0.0 0.115; H 0.0 0.754 -0.459; H 0.0 -0.754 -0.459",
    # atom=open('PSPCz.xyz').read(),
   atom=open('BODIPY.xyz').read(),
   basis="6-31g*",
    method=MethodType.RKS,
    xc_functional=f"ldaerf + lr_hf({omega})",
    xcf_library="xcfun",
)

In [ ]:
'''
1. Run the reference calculation
'''
driver.run_pyscf()
mf = driver._calc

In [3]:
print(f'Le nombre total d\'électrons est {driver._mol.nelectron} \
et le nombre total d\'électrons (alpha, beta) est {driver._mol.nelec}\n')
print(f'Les index (0-Based) du (HOMO,LUMO) sont {driver._mol.nelectron//2 -1, driver._mol.nelectron//2}\n')
print(f'Le nombre d\'orbitales atomiques, dans la base {driver._mol.basis}, est {driver._mol.nao}\n')
print(f'L\'énergie nucléaire vaut {driver._mol.energy_nuc()} Ha')

AttributeError: 'NoneType' object has no attribute 'nelectron'

In [ ]:
import plotly.express as px

# Plot the MO Occupations
fig = px.line(y=mf.mo_occ, markers=True, title="Molecular Orbital (MO) Occupations")
fig.update_layout(xaxis_title="Orbital Index (0-Based)", yaxis_title="MO Occupation")
fig.show()

In [ ]:
from pyscf.data.nist import HARTREE2EV as au2ev 

lumo_idx = mf.mo_occ.tolist().index(0.)
homo_idx = lumo_idx - 1
print(f'Energie du Homo = {mf.mo_energy[homo_idx] * au2ev} eV')
print(f'Energie de Lumo = {mf.mo_energy[lumo_idx] * au2ev} eV')
print(f'Energie du gap Homo-Lumo = {abs(mf.mo_energy[homo_idx] - mf.mo_energy[lumo_idx]) * au2ev} eV')

Energie du Homo = -8.091224328787915 eV
Energie de Lumo = -0.28901488413575743 eV
Energie du gap Homo-Lumo = 7.802209444652157 eV


In [ ]:
# Plot the MO Energies (i.e. eigenvalues of the Fock matrix)
fig = px.line(y=mf.mo_energy, markers=True, title="Molecular Orbital (MO) Energies (a.u.)")
fig.update_layout(xaxis_title="Orbital Index (0-Based)", yaxis_title="MO Energies (a.u.)")
fig.show()

In [ ]:
mf.dump_scf_summary()

**** SCF Summaries ****
Total Energy =                        -676.198982803146691
Nuclear Repulsion Energy =             858.640651535210282
One-electron Energy =                -2636.450006542323990
Two-electron Coulomb Energy =         1187.014393483860204
DFT Exchange-Correlation Energy =      -85.404021279893172


## ElectronicEnergy calculation

La classe `ElectronicEnergy` implémente le Hamiltonien 

$$ H_{el} = \sum_{p, q} h_{pq} a^\dagger_p a_q
         + \sum_{p, q, r, s} g_{pqrs} a^\dagger_p a^\dagger_q a_r a_s ,$$

où $h_{pq}$ et $g_{pqrs}$ sont les intégrales électroniques à un et deux corps,
stocké dans le conteneur `qiskit_nature.second_q.operators.ElectronicIntegrals`.
Lorsqu'il s'agit de coefficients séparés pour les électrons de spin $\alpha$- et $\beta$,
l'Hamiltonien à spin sans restriction (UHF) peut être obtenu à partir de celui ci-dessus de manière simple.

On peut construire une instance de cet Hamiltonien de plusieurs manières.

1. Avec une instance existante de `qiskit_nature.second_q.operators.ElectronicIntegrals` :

```python
   intégrales : ElectronicIntegrals = ...

   hamiltonien = ElectronicEnergy (intégrales, constantes = {"nuclear_repulsion_energy": 1.0})
```
2. À partir d’un ensemble brut de matrices de coefficients intégraux 
    * h1_a: the alpha-spin one-body coefficients.
    * h2_aa: the alpha-alpha-spin two-body coefficients.
    * h1_b: the beta-spin one-body coefficients.
    * h2_bb: the beta-beta-spin two-body coefficients.
    * h2_ba: the beta-alpha-spin two-body coefficients.:

```python
    # en supposant que les intégrales à un et deux corps ont été préalablement calculées
    h1_a, h2_aa, h1_b, h2_bb, h2_ba = ...

    hamiltonien = ElectronicEnergy.from_raw_integrals(h1_a, h2_aa, h1_b, h2_bb, h2_ba)
    hamiltonian.nuclear_repulsion_energy = 1.0
```
3. À partir d'un instance QSChema
```python
    from qiskit_nature.second_q.formats.qcschema import QCSchema
    from qiskit_nature.second_q.formats import qcschema_translator 

    qcschema = driver.to_qcschema(include_dipole=True)
    hamiltonian = qcschema_translator._get_mo_hamiltonian_direct(qcschema)

```

Il est à noter que nous avons spécifié l'énergie de répulsion nucléaire comme un décalage d'énergie constant dans ce qui précède. Ce terme ne sera pas inclus dans l'opérateur qubit mappé puisqu'il s'agit d'un terme de décalage constant et qu'il ne nécessite pas d'erreurs lors de la mesure sur un appareil quantique. Il est cependant possible d'inclure des termes d'énergie constante à l'intérieur du conteneur `qiskit_nature.second_q.operators.ElectronicIntegrals`, si l'on souhait qu'il soit inclus dans l'opérateur qubit, lors du mapping de l'opérateur de 2e quantification à l'espace qubit.
 ``` python
    from qiskit_nature.second_q.operators import PolynomialTensor

        e_nuc = hamiltonian.nuclear_repulsion_energy
        hamiltonian.electronic_integrals.alpha += PolynomialTensor({"": e_nuc})
        hamiltonian.nuclear_repulsion_energy = None
```

In [ ]:
# ''' 
# 2. Calculate the h1e and h2e integrals
# '''
# import numpy as np
# from pyscf import ao2mo 
# from qiskit_nature.second_q.operators.symmetric_two_body import fold


# mo_coeff, mo_coeff_b = driver._expand_mo_object(mf.mo_coeff, array_dimension=3)
# h1_a = mf.get_hcore()
# h1_a = np.dot(np.dot(mo_coeff.T, h1_a), mo_coeff)
# if mo_coeff_b is not None:
#     h1_b = np.dot(np.dot(mo_coeff_b.T, h1_a), mo_coeff_b)
# else:
#     h1_b = None


# h2_aa = driver._mol.intor("int2e", aosym=8)
# h2_aa = fold(ao2mo.full(driver._mol, mo_coeff, aosym=4))
# if mo_coeff_b is not None:
#     h2_bb = fold(ao2mo.full(driver._mol, mo_coeff_b, aosym=4))
#     h2_ba = fold(ao2mo.general(
#             driver._mol,
#             [mo_coeff_b, mo_coeff_b, mo_coeff, mo_coeff],
#             aosym=4,
#         )
#     )
# else:
#     h2_bb = None
#     h2_ba = None

In [ ]:
# '''
#  3. Build the total Hamiltonian outside of a Problem instance
# '''
# # from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
# # from qiskit_nature.second_q.operators import ElectronicIntegrals, PolynomialTensor

# # hamiltonian = ElectronicEnergy.from_raw_integrals(
# #     h1_a,
# #     h2_aa,
# #     h1_b,
# #     h2_bb,
# #     h2_ba
# # )
'''
 2. Build the total Hamiltonian outside of a Problem instance
 Use of QCSchema
'''
from qiskit_nature.second_q.formats.qcschema import QCSchema
from qiskit_nature.second_q.formats import qcschema_translator 
from qiskit_nature.second_q.operators import PolynomialTensor

qcschema = driver.to_qcschema(include_dipole=True)
hamiltonian = qcschema_translator._get_mo_hamiltonian_direct(qcschema)
# print(hamiltonian.second_q_op())

hamiltonian.nuclear_repulsion_energy = driver._mol.energy_nuc()

# To included the constant nuclear_repulsion_energy in the second_q_op
# e_nuc = hamiltonian.nuclear_repulsion_energy
# hamiltonian.electronic_integrals.alpha += PolynomialTensor({"": e_nuc})
# hamiltonian.nuclear_repulsion_energy = None
# print(hamiltonian.second_q_op())

In [1]:
# To check that our defined Hamiltonian correct
# print(driver.run().hamiltonian.second_q_op())
# hamiltonian.second_q_op() == driver.run().hamiltonian.second_q_op()

## Active-Space reduction

L'énergie électronique totale est définie par, 
$$
    E = \sum_{pq} h_{pq}D_{pq} + \frac12\sum_{pqrs} g_{pqrs}d_{pqrs},
$$
où $h_{pq}$ et $g_{pqrs}$ sont les intégrales à un et deux électrons, respectivement, et $D$ et $d$ sont les opérateurs statistiques (ou matrices de densité) à une et deux particules.

Pour définir un espace actif, nous divisons l'opérateur à un électron en une partie active et une partie inactive, $D=D^A+D^I$. Dans la base des MOs, cette dernière se simplifie à $D^I_{iq}=2\delta_{iq}$, où nous utilisons la notation des indices de Helgaker dans laquelle $i,j,k,l$ désignent *inactif*, $u,v,x,y$ désignent *actif* et $p,q,r,s$ désignent les MOs *général*. On obtient ainsi,
$$
    E = E^I + \sum_{uv} F^I_{uv} D_{uv}^A + \frac12 \sum_{uvxy} g_{uvxy}d_{uvxy}^A,
$$
où l'**opérateur de Fock inactif** est définie comme
$$ F^I_{pq} = h_{pq} + \sum_i (2 g_{iipq} - g_{iqpi}),$$
et l'**énergie inactive** est donnée par
$$ E^I = \sum_j h_{ji} + F^I_{jj} = \frac12 \sum_{ij} \Big(h_{ij} + F^I_{jj}\Big) D^I_{ij} .$$

C'est ce qui est implémenté dans `qiskit_nature.second_q.transformers.ActiveSpaceTransformer`.

On note que 
* l'opérateur de Fock *inactive* L'opérateur de Fock, $F^I$, remplace les intégrales à un électron, $h_{pq}$, 
* les opérateurs statistiques à un et deux électrons, $D^A$ et $d^A$, remplacent $D$ et $d$, et le décalage d'énergie constante, $E^I$, est ajouté.

Par conséquent, l'Hamiltonien simulé dans le calculateur quantique  la forme suivante
$$
    \mathtt{H}_{qc} = \sum_{uv} F^I_{uv} a^\dagger_u a_v + \sum_{uvxy} g_{uvxy} 
    a^\dagger_u a^\dagger_v a_x a_y .
$$
En utilisant l'opérateur de Fock inactif à la place des intégrales à un électron, la description de l'espace actif contient un potentiel effectif généré par les électrons inactifs. Par conséquent, cette méthode permet l'exclusion des électrons non centraux tout en conservant une description de haute qualité du système.

On rappelle que l'opérateur de Fock est défini comme
$$ F_{pq} = h_{pq} + J_{pq} - K_{pq},$$
avec $J_{qr} = \sum g_{pqrs} D_{ps}$ et $K_{pr} = \sum g_{pqrs} D_{qs}$ est opérateurs de Coulomb et d'échange respectivement.

In [16]:
''' 
4. Prepare the active space by initializing it with the total problem size information
'''
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
import numpy as np

# Somes variables needed
total_num_particles = driver._mol.nelec
total_num_spatial_orbitals = driver._mol.nao # hamiltonian.register_length
total_num_electrons = driver._mol.nelectron

# Alpha-spin
num_alpha = total_num_particles[0] # qcschema.properties.calcinfo_nalpha
orbital_occupations = np.asarray([1.0] * num_alpha 
                                 + [0.0] * (total_num_spatial_orbitals - num_alpha))
# Beta-spin
num_beta = total_num_particles[1] # , qcschema.properties.calcinfo_nbeta
orbital_occupations_b = np.asarray([1.0] * num_beta 
                                   + [0.0] * (total_num_spatial_orbitals - num_beta))
# Initialize the transformer
transformer =  ActiveSpaceTransformer(2,2)

# Prepare the active space
transformer.prepare_active_space(total_num_particles, total_num_spatial_orbitals,
                                 occupation_alpha=orbital_occupations, 
                                 occupation_beta=orbital_occupations_b,)
# Determine the active space
as_orbitals = transformer._determine_active_space(total_num_electrons, total_num_spatial_orbitals)[0]

# Use the active space transformer to reduce the total Hamiltonian to the active space
reduced_hamiltonian = transformer.transform_hamiltonian(hamiltonian)

: 

In [ ]:
print(as_orbitals)
print(reduced_hamiltonian.second_q_op())

[4, 5]
Fermionic Operator
number spin orbitals=4, number terms=36
  0.3861043049679476 * ( +_0 +_0 -_0 -_0 )
+ 0.1866879907489356 * ( +_0 +_1 -_1 -_0 )
+ 0.3861043049679476 * ( +_0 +_2 -_2 -_0 )
+ 0.1866879907489356 * ( +_0 +_3 -_3 -_0 )
+ 0.006783659597588433 * ( +_0 +_0 -_1 -_1 )
+ 0.006783659597588433 * ( +_0 +_1 -_0 -_1 )
+ 0.006783659597588433 * ( +_0 +_2 -_3 -_1 )
+ 0.006783659597588433 * ( +_0 +_3 -_2 -_1 )
+ 0.006783659597588433 * ( +_1 +_0 -_1 -_0 )
+ 0.006783659597588433 * ( +_1 +_1 -_0 -_0 )
+ 0.006783659597588433 * ( +_1 +_2 -_3 -_0 )
+ 0.006783659597588433 * ( +_1 +_3 -_2 -_0 )
+ 0.1866879907489356 * ( +_1 +_0 -_0 -_1 )
+ 0.16964829513111165 * ( +_1 +_1 -_1 -_1 )
+ 0.1866879907489356 * ( +_1 +_2 -_2 -_1 )
+ 0.16964829513111165 * ( +_1 +_3 -_3 -_1 )
+ 0.3861043049679476 * ( +_2 +_0 -_0 -_2 )
+ 0.1866879907489356 * ( +_2 +_1 -_1 -_2 )
+ 0.3861043049679476 * ( +_2 +_2 -_2 -_2 )
+ 0.1866879907489356 * ( +_2 +_3 -_3 -_2 )
+ 0.006783659597588433 * ( +_2 +_0 -_1 -_3 )
+ 0.0067836

In [ ]:
from pyscf.tools import cubegen

# Output cube files for active orbitals that can read by Jmol. Avogadro, Gabedit
for i in as_orbitals:
    cubegen.orbital(
        driver._mol, 
        f'.Test_DFT_{i+1}.cube', 
        mf.mo_coeff[:, i])

In [ ]:
''' 
5. Setup the Problem instance
'''
from qiskit_nature.second_q.problems import ElectronicBasis, ElectronicStructureProblem

reduced_hamiltonian.constants["inactive energy"] = mf.e_tot - driver._mol.energy_nuc()

problem = ElectronicStructureProblem(reduced_hamiltonian)
problem.basis = ElectronicBasis.MO
problem.num_spatial_orbitals = transformer._num_spatial_orbitals
problem.num_particles = (transformer._num_electrons // 2, transformer._num_electrons // 2)
problem.orbital_energies = mf.mo_energy[as_orbitals] 
problem.reference_energy = mf.e_tot



In [ ]:
''' 
Problem Electronic properties
'''
from qiskit_nature.second_q.properties import (
    AngularMomentum,
    ElectronicDensity,
    Magnetization,
    ParticleNumber,
)

norb = len(as_orbitals) # number of active orbitals

#FIXME
# nao = hamiltonian.register_length
# overlap = qcschema_translator._reshape_2(qcschema.wavefunction.scf_overlap, nao, nao)

problem.properties.particle_number = ParticleNumber(norb)
problem.properties.magnetization = Magnetization(norb)
problem.properties.angular_momentum = AngularMomentum(norb)
problem.properties.electronic_density = ElectronicDensity.from_orbital_occupation(
    problem.orbital_occupations, 
    problem.orbital_occupations_b
)

In [ ]:
# Electronic dipole moment
from qiskit_nature.second_q.properties import ElectronicDipoleMoment

dipole_x = qcschema_translator._get_mo_dipole(qcschema, "x")
dipole_y = qcschema_translator._get_mo_dipole(qcschema, "y")
dipole_z = qcschema_translator._get_mo_dipole(qcschema, "z")

dipole = ElectronicDipoleMoment(dipole_x, dipole_y, dipole_z)
if qcschema.properties.nuclear_dipole_moment is not None:
    dipole.nuclear_dipole_moment = qcschema.properties.nuclear_dipole_moment

problem.properties.electronic_dipole_moment = dipole

In [ ]:
''' 
6. mapping the Hamiltonian to spin space
'''
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper, TaperedQubitMapper

mapper = ParityMapper(num_particles=problem.num_particles)
mapper = problem.get_tapered_mapper(mapper)

Red_hamil_z2qubit = mapper.map(reduced_hamiltonian.second_q_op())
print(f"Number of items in the PM Z2 Pauli list:", len(Red_hamil_z2qubit))
Red_hamil_z2qubit


Number of items in the PM Z2 Pauli list: 3


SparsePauliOp(['I', 'Z', 'X'],
              coeffs=[-1.20620975+0.j, -0.52247029+0.j, -0.01356732+0.j])

In [ ]:
from qiskit_nature.second_q.circuit.library import HartreeFock

In [ ]:
''' 
5. Setup solver 
'''
from pyscf import fci
from qiskit_nature_pyscf import PySCFGroundStateSolver

solver_fci = PySCFGroundStateSolver(fci.direct_spin1.FCI())

result_fci = solver_fci.solve(problem)
print(result_fci)

RuntimeError: eri.size = 48, norb = 2

In [ ]:
''' 
7. Setup solver 
'''
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_algorithms import NumPyEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# problem = driver.run()

solver = NumPyMinimumEigensolver(filter_criterion=problem.get_default_filter_criterion())

algo = GroundStateEigensolver(mapper, solver)
result = algo.solve(problem)

print(f"Total ground state energy = {result.total_energies[0]:.4f}")



AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
from qiskit_algorithms import NumPyEigensolver
from qiskit_nature.second_q.algorithms import ExcitedStatesEigensolver

# Setup the solver
numpy_solver = NumPyEigensolver(k=4, filter_criterion=problem.get_default_filter_criterion())

numpy_ES_solver = ExcitedStatesEigensolver(mapper, numpy_solver)
NP_ES = numpy_ES_solver.solve(problem)

print(NP_ES)



AttributeError: 'NoneType' object has no attribute 'get'